In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly as plotly
from plotly.offline import plot,iplot,init_notebook_mode,download_plotlyjs
import datetime as datetime
import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline()
init_notebook_mode(connected=True)
%matplotlib inline

In [3]:
fulldb = pd.read_excel('MPVDatasetDownload.xlsx')
fulldb['Year'] = fulldb['Date of Incident (month/day/year)'].apply(lambda x:x.year)
fulldb['Month'] = fulldb['Date of Incident (month/day/year)'].apply(lambda x:x.month)
fulldb = fulldb[(fulldb["Victim's race"] != 'Unknown race') & (fulldb["Victim's race"] != 'Unknown Race')]
fulldb = fulldb[fulldb["Victim's name"] != 'Name withheld by police']
fulldb = fulldb[(fulldb["Victim's gender"] != 'Unknown') & (fulldb["Victim's gender"] != 'Transgender')]
fulldb["Victim's age"] = fulldb["Victim's age"].apply(lambda x:x if type(x)== int else 'Unknown')
fulldb = fulldb[fulldb["Victim's age"] != 'Unknown']

In [130]:
#SEX
data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        data_each_yr = dict(
                            type='bar',
                            x = fulldb["Victim's gender"].value_counts().index,
                            y=fulldb[fulldb['Year'] == year]["Victim's gender"].value_counts(),
                            hovertemplate = '<b><i>Sex: %{x}</i></b><br><br>Number of Deaths: %{y}<br>Proportion of ' + str(year)  +  ' Total: ' + (round((fulldb[fulldb['Year']==year]["Victim's gender"].value_counts()/len(fulldb[fulldb['Year']==year]["Victim's gender"]))*100,2)).astype(str) + '%<extra></extra>',
                            name = year,
                            marker_color='#22c94f',
                            marker_line_color='black')
    else:
        data_each_yr = dict(
                            type='bar',
                            x = fulldb["Victim's gender"].value_counts().index,
                            y=fulldb["Victim's gender"].value_counts(),
                            hovertemplate = '<b><i>Sex: %{x}</i></b><br><br>Number of Deaths: %{y}<br>Proportion of Total: ' + (round((fulldb["Victim's gender"].value_counts()/len(fulldb["Victim's gender"]))*100,2)).astype(str) + '%<extra></extra>',
                            name = '2013-2019 Total',
                            marker_color='#22c94f',
                            marker_line_color='black')
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 20}, steps=steps,bgcolor='#22c94f',bordercolor='black')]

layout = dict(title ={'text':'Sex','x':0.5,'y':0.8},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=400)

fig = go.Figure(dict(data=data_slider, layout=layout))

fig.show()

data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        data_each_yr = dict(
                            type='pie',
                            labels = fulldb["Victim's gender"].value_counts().index,
                            values=fulldb[fulldb['Year'] == year]["Victim's gender"].value_counts(),
                            hovertemplate = '<b><i>Sex: ' + fulldb["Victim's gender"].value_counts().index +'</i></b><br><br>Number of Deaths: ' + fulldb[fulldb['Year'] == year]["Victim's gender"].value_counts().astype(str) + '<br>Percentage of ' + str(year)  +  ' Total: %{percent}%<extra></extra>',
                            name = year,
                            marker = dict(colors=['#FAAF40','BE1E20'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=14,
                            outsidetextfont_size=14
                            )
    else:
        data_each_yr = dict(
                            type='pie',
                            labels = fulldb["Victim's gender"].value_counts().index,
                            values=fulldb["Victim's gender"].value_counts(),
                            hovertemplate = '<b><i>Sex: ' + fulldb["Victim's gender"].value_counts().index +'</i></b><br><br>Number of Deaths: ' + fulldb["Victim's gender"].value_counts().astype(str)+ '<br>Percentage of Total: %{percent}%<extra></extra>',
                            name = '2013-2019 Cumulative Total',
                            marker = dict(colors=['#FAAF40','BE1E20'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=14,
                            outsidetextfont_size=14)
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Cumulative Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 0}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]

layout = dict(title ={'text':'Sex','x':0.47,'y':0.85},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=400,showlegend=True)

figsex = go.Figure(dict(data=data_slider, layout=layout))

figsex.show()

In [152]:
#AGE GROUP
data_slider = []
index = ['0-14','15-24','25-54','55-64','64+']
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        df = fulldb[fulldb['Year'] == year]
        y = [len(df[df["Victim's age"] <= 14]),len(df[(df["Victim's age"] >= 15) & (df["Victim's age"] <= 24)]),len(df[(df["Victim's age"] >= 25) & (df["Victim's age"] <= 54)]),len(df[(df["Victim's age"] >= 55) & (df["Victim's age"] <= 64)]),len(df[df["Victim's age"] >= 65])]
        newdf = pd.DataFrame()
        newdf['y'] = y
        data_each_yr = dict(
                            type='bar',
                            x=index,
                            y = y,
                            hovertemplate = '<b><i>Age Group: %{x}</i></b><br><br>Number of Deaths: %{y}<br>Proportion of ' + str(year)  +  ' Total: ' + (round(100*newdf['y']/sum(y),2)).astype(str) + '%<extra></extra>',
                            name = year,
                            marker_color='#22c94f',
                            marker_line_color='black')
    else:
        df = fulldb
        y = [len(df[df["Victim's age"] <= 14]),len(df[(df["Victim's age"] >= 15) & (df["Victim's age"] <= 24)]),len(df[(df["Victim's age"] >= 25) & (df["Victim's age"] <= 54)]),len(df[(df["Victim's age"] >= 55) & (df["Victim's age"] <= 64)]),len(df[df["Victim's age"] >= 65])]
        newdf = pd.DataFrame()
        newdf['y'] = y
        data_each_yr = dict(
                            type='bar',
                            x=index,
                            y = y,
                            hovertemplate = '<b><i>Age Group: %{x}</i></b><br><br>Number of Deaths: %{y}<br>Proportion of ' + str(year)  +  ' Total: ' + (round(100*newdf['y']/sum(y),2)).astype(str) + '%<extra></extra>',
                            name = year,
                            marker_color='#22c94f',
                            marker_line_color='black')
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 20}, steps=steps,bgcolor='#22c94f',bordercolor='black')]

layout = dict(title ={'text':'Age Group','x':0.5,'y':0.8},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=400)

fig1 = go.Figure(dict(data=data_slider, layout=layout))

data_slider = []
index = ['0-14','15-24','25-54','55-64','65+']
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        df = fulldb[fulldb['Year'] == year]
        y = [len(df[df["Victim's age"] <= 14]),len(df[(df["Victim's age"] >= 15) & (df["Victim's age"] <= 24)]),len(df[(df["Victim's age"] >= 25) & (df["Victim's age"] <= 54)]),len(df[(df["Victim's age"] >= 55) & (df["Victim's age"] <= 64)]),len(df[df["Victim's age"] >= 65])]
        newdf = pd.DataFrame()
        newdf['y'] = y
        newdf['x'] = index
        data_each_yr = dict(
                            type='pie',
                            labels=index,
                            values = y,
                            hovertemplate = '<b><i>Age Group: '+ newdf['x'].astype(str) + '</i></b><br><br>Number of Deaths: '+newdf['y'].astype(str)+'<br>Percentage of ' + str(year)  +  ' Total: %{percent}%<extra></extra>',
                            name = year,
                            marker = dict(colors=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=12,
                            outsidetextfont_size=12)
    else:
        df = fulldb
        y = [len(df[df["Victim's age"] <= 14]),len(df[(df["Victim's age"] >= 15) & (df["Victim's age"] <= 24)]),len(df[(df["Victim's age"] >= 25) & (df["Victim's age"] <= 54)]),len(df[(df["Victim's age"] >= 55) & (df["Victim's age"] <= 64)]),len(df[df["Victim's age"] >= 65])]
        newdf = pd.DataFrame()
        newdf['y'] = y
        newdf['x'] = index
        data_each_yr = dict(
                            type='pie',
                            labels=index,
                            values = y,
                            hovertemplate = '<b><i>Age Group: '+newdf['x'].astype(str)+'</i></b><br><br>Number of Deaths: '+newdf['y'].astype(str)+'<br>Percentage of Total: %{percent}%<extra></extra>',
                            name = year,
                            marker = dict(colors=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=12,
                            outsidetextfont_size=12)
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Cumulative Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 0}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]

layout = dict(title ={'text':'Age Group','x':0.5,'y':0.85},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=400,showlegend=True)

figage = go.Figure(dict(data=data_slider, layout=layout))

fig1.show()
figage.show()

In [145]:
#RACE
data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        data_each_yr = dict(
                            type='bar',
                            x = fulldb["Victim's race"].value_counts().index,
                            y=fulldb[fulldb['Year'] == year]["Victim's race"].value_counts(),
                            hovertemplate = '<b><i>Race: %{x}</i></b><br><br>Number of Deaths: %{y}<br>Proportion of ' + str(year)  +  ' Total: ' + (round((fulldb[fulldb['Year']==year]["Victim's race"].value_counts()/len(fulldb[fulldb['Year']==year]["Victim's race"]))*100,2)).astype(str) + '%<extra></extra>',
                            name = year,
                            marker_color='#22c94f',
                            marker_line_color='black')
    else:
        data_each_yr = dict(
                            type='bar',
                            x = fulldb["Victim's race"].value_counts().index,
                            y=fulldb["Victim's race"].value_counts(),
                            hovertemplate = '<b><i>Race: %{x}</i></b><br><br>Number of Deaths: %{y}<br>Proportion of Total: ' + (round((fulldb["Victim's race"].value_counts()/len(fulldb["Victim's race"]))*100,2)).astype(str) + '%<extra></extra>',
                            name = '2013-2019 Total',
                            marker_color='#22c94f',
                            marker_line_color='black')
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 30}, steps=steps,bgcolor='#22c94f',bordercolor='black')]

layout = dict(title ={'text':'Race','x':0.5,'y':0.8},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=400)

fig2 = go.Figure(dict(data=data_slider, layout=layout))

fig2.show()


data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        data_each_yr = dict(
                            type='pie',
                            labels = fulldb["Victim's race"].value_counts().index,
                            values=fulldb[fulldb['Year'] == year]["Victim's race"].value_counts(),
                            hovertemplate = '<b><i>Race: '+ fulldb["Victim's race"].value_counts().index +'</i></b><br><br>Number of Deaths: '+ fulldb[fulldb['Year'] == year]["Victim's race"].value_counts().astype(str) +'<br>Percentage of ' + str(year)  +  ' Total: %{percent}%<extra></extra>',
                            name = year,
                            marker = dict(colors=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28','#262261'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=12,
                            outsidetextfont_size=12)
    else:
        data_each_yr = dict(
                            type='pie',
                            labels = fulldb["Victim's race"].value_counts().index,
                            values=fulldb["Victim's race"].value_counts(),
                            hovertemplate = '<b><i>Race: ' + fulldb["Victim's race"].value_counts().index + '</i></b><br><br>Number of Deaths: '+ fulldb["Victim's race"].value_counts().astype(str) +'<br>Percentage of Total: %{percent}%<extra></extra>',
                            name = '2013-2019 Total',
                            marker = dict(colors=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28','#262261'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=12,
                            outsidetextfont_size=12)
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Cumulative Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 0}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]


layout = dict(title ={'text':'Race','x':0.43,'y':0.85},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=500,showlegend=True)

figrace = go.Figure(dict(data=data_slider, layout=layout))

figrace.show()

In [138]:
#STATE
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
codetostate = {}
for key,value in us_state_abbrev.items():
    codetostate[value] = key
    
    
    
fulldb.sort_values('Date of Incident (month/day/year)',inplace=True)
year = 2013

scl = [[0.0, '#00A69C'],[0.5, '#FAAF40'],[1.0, '#BE1E20']]

data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019]:
    df_segmented = pd.DataFrame()
    df_segmented =  fulldb[fulldb['Year']<= year]['State'].value_counts().to_frame()
    df_segmented.columns = ['Count']

    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented.index,
                        z=(df_segmented['Count']-df_segmented['Count'].mean())/df_segmented['Count'].std(),
                        locationmode='USA-states',
                        colorscale = scl,
                        hovertemplate = '<b><i>State: '+ df_segmented.index.map(codetostate) + '</i></b><br><br>Number of Deaths: ' + df_segmented['Count'].astype(str) + '<br>Percentage of Total: ' + round(100*(df_segmented['Count']/len(fulldb[fulldb['Year']<= year]['State'])),2).astype(str) + '%<extra></extra>',
                        showscale=False)

    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2013))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 0}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]

layout = dict(title ={'text':'Cumulative Police Killings by State','x':0.5,'y':0.8}, geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders,height=400,width=650)

fig3 = dict(data=data_slider, layout=layout)
fig3=go.Figure(fig3)
fig3.show()

In [146]:
#RACE X SEX
racelist = ['White','Black','Hispanic','Asian','Native American','Pacific Islander']
sexlist = ['Male','Female']

racecol1 = []
gendercol1 = []
yearcol = []
list1 = []
for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
    if type(year) == int or type(year) == float:
        trialdf = fulldb[fulldb['Year'] == year]
    else:
        trialdf = fulldb
    for race in racelist:
        for sex in sexlist:
            racecol1.append(race)
            gendercol1.append(sex)
            yearcol.append(year)
            list1.append(len(trialdf[(trialdf["Victim's race"] == race) & (trialdf["Victim's gender"] == sex)]))

            
df1 = pd.DataFrame()
df1['Race'] = racecol1
df1['Sex'] = gendercol1
df1['Year'] = yearcol
df1['Deaths'] = list1

data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
    data_each_yr = dict(
                        type='bar',
                        x = df1[(df1['Year'] == year) & (df1['Sex'] == 'Male')]['Race'],
                        y=df1[(df1['Year'] == year) & (df1['Sex'] == 'Male')]['Deaths'],
                        hovertemplate = '<b><i>' + str(year) +': Race -- %{x}, Sex -- Male</i></b><br><br>Number of Deaths: %{y}',
                        name = 'Male',
                        marker_color='#FAAF40',
                        marker_line_color='black')
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Cumulative Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 30}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]


layout = dict(title ={'text':'Race x Sex','x':0.5,'y':0.8},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=650)

figracesex = go.Figure(dict(data=data_slider, layout=layout))

for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
    figracesex.add_trace(go.Bar(
    x = df1[(df1['Year'] == year) & (df1['Sex'] == 'Female')]['Race'],
    y=df1[(df1['Year'] == year) & (df1['Sex'] == 'Female')]['Deaths'],
    hovertemplate = '<b><i>' + str(year) +': Race -- %{x}, Sex -- Male</i></b><br><br>Number of Deaths: %{y}',
    name = 'Female',
    marker_color='#BE1E20',
    marker_line_color='black'))


figracesex.update_layout(plot_bgcolor='#f0f2f7',paper_bgcolor='white')
figracesex.show()


In [149]:
#RACE X AGE GROUP
racelist = ['White','Black','Hispanic','Asian','Native American','Pacific Islander']
agelist = [14,24,54,64]

racecol = []
agecol = []
yearcol = []
deathcol = []
for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
    if type(year) == int or type(year) == float:
        trialdf = fulldb[fulldb['Year'] == year]
    else:
        trialdf = fulldb
    for race in racelist:
        for num in range(0,len(agelist)+1):
            if num == 0:
                racecol.append(race)
                agecol.append('0-14')
                yearcol.append(year)
                deathcol.append(len(trialdf[(trialdf["Victim's race"] == race) & (trialdf["Victim's age"] <= 14)]))
            elif num < 4:
                racecol.append(race)
                agecol.append(str(agelist[num-1]+1) + '-' + str(agelist[num]))
                yearcol.append(year)
                deathcol.append(len(trialdf[(trialdf["Victim's race"] == race) & (trialdf["Victim's age"] > agelist[num-1]) & (trialdf["Victim's age"] <= agelist[num])]))
            else:
                racecol.append(race)
                agecol.append('65+')
                yearcol.append(year)
                deathcol.append(len(trialdf[(trialdf["Victim's race"] == race) & (trialdf["Victim's age"] > 64)]))
        

df = pd.DataFrame()
df['Race'] = racecol
df['Age'] = agecol
df['Year'] = yearcol
df['Deaths'] = deathcol

data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
    data_each_yr = dict(
                        type='bar',
                        x = df[(df['Year'] == year) & (df['Age'] == '0-14')]['Race'],
                        y=df[(df['Year'] == year) & (df['Age'] == '0-14')]['Deaths'],
                        hovertemplate = '<b><i>' + str(year) +': Race -- %{x}, Age Group -- 0-14</i></b><br><br>Number of Deaths: %{y}',
                        name = '0-14',
                        marker_color='#e68f43',
                        marker_line_color='black')
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Cumulative Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 30}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]

layout = dict(title ={'text':'Race x Age','x':0.5,'y':0.8},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=650)

figraceage = go.Figure(dict(data=data_slider, layout=layout))

agelist1 = ['15-24','25-54','55-64','65+']
for age in agelist1:
    for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
        figraceage.add_trace(go.Bar(
        x = df[(df['Year'] == year) & (df['Age'] == age)]['Race'],
        y = df[(df['Year'] == year) & (df['Age'] == age)]['Deaths'],
        name = age,
        marker_line_color = 'black',
        hovertemplate = '<b><i>'+ str(year) + ': Race -- %{x}, Age -- ' + age + '</b></i><br><br>Number of Deaths %{y}'
        ))

figraceage.update_layout(colorway=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28'],plot_bgcolor='#f0f2f7',paper_bgcolor='white')    
figraceage.show()                   

In [150]:
#AGE GROUP X SEX
sexlist = ['Male','Female']
agelist = [14,24,54,64]

sexcol = []
agecol = []
yearcol = []
deathcol = []

for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
    if type(year) == int or type(year) == float:
        trialdf = fulldb[fulldb['Year'] == year]
    else:
        trialdf = fulldb
    for sex in sexlist:
        for num in range(0,len(agelist)+1):
            if num == 0:
                sexcol.append(sex)
                agecol.append('0-14')
                yearcol.append(year)
                deathcol.append(len(trialdf[(trialdf["Victim's gender"] == sex) & (trialdf["Victim's age"] <= 14)]))
            elif num < 4:
                sexcol.append(sex)
                agecol.append(str(agelist[num-1]+1) + '-' + str(agelist[num]))
                yearcol.append(year)
                deathcol.append(len(trialdf[(trialdf["Victim's gender"] == sex) & (trialdf["Victim's age"] > agelist[num-1]) & (trialdf["Victim's age"] <= agelist[num])]))
            else:
                sexcol.append(sex)
                agecol.append('65+')
                yearcol.append(year)
                deathcol.append(len(trialdf[(trialdf["Victim's gender"] == sex) & (trialdf["Victim's age"] > 64)]))
                
df = pd.DataFrame()
df['Sex'] = sexcol
df['Age'] = agecol
df['Year'] = yearcol
df['Deaths'] = deathcol

data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
    data_each_yr = dict(
                        type='bar',
                        x = df[(df['Year'] == year) & (df['Sex'] == 'Male')]['Age'],
                        y=df[(df['Year'] == year) & (df['Sex'] == 'Male')]['Deaths'],
                        hovertemplate = '<b><i>' + str(year) +': Age Group -- %{x}, Sex -- Male</i></b><br><br>Number of Deaths: %{y}',
                        name = 'Male',
                        marker_color='#FAAF40',
                        marker_line_color='black')
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Cumulative Total')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 30}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]

layout = dict(title ={'text':'Age x Sex','x':0.5,'y':0.8},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=650)

figagesex = go.Figure(dict(data=data_slider, layout=layout))

for year in [2013,2014,2015,2016,2017,2018,2019,'2013-2019 Cumulative Total']:
    figagesex.add_trace(go.Bar(
    x = df[(df['Year'] == year) & (df['Sex'] == 'Female')]['Age'],
    y=df[(df['Year'] == year) & (df['Sex'] == 'Female')]['Deaths'],
    hovertemplate = '<b><i>' + str(year) +': Age Group -- %{x}, Sex -- Female</i></b><br><br>Number of Deaths: %{y}',
    name = 'Female',
    marker_color='#BE1E20',
    marker_line_color='black'
    ))

figagesex.update_layout(plot_bgcolor='#f0f2f7',paper_bgcolor='white')
figagesex.show()


In [13]:
yeardata = pd.read_excel('nst-est2019-01.xlsx')
yeardata.columns = yeardata.iloc[0]
yeardata.drop([0,1,2,3,4,5,57,59,60,61,62,63],axis=0,inplace=True)
newcols = []
for cols in yeardata.columns:
    if type(cols) == np.float64:
        newcols.append(int(cols))
    else:
        newcols.append(cols)
yeardata.columns = newcols
yeardata['Geographic Area'] = yeardata['Geographic Area'].apply(lambda x:x.split('.')[1] if '.' in x else x)
yeardata['State Code'] = yeardata['Geographic Area'].map(us_state_abbrev)
yeardata

,Geographic Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,State Code
6,Alabama,4779736,4780125,4785437.0,4799069.0,4815588.0,4830081.0,4841799.0,4852347.0,4863525.0,4874486.0,4887681.0,4903185.0,AL
7,Alaska,710231,710249,713910.0,722128.0,730443.0,737068.0,736283.0,737498.0,741456.0,739700.0,735139.0,731545.0,AK
8,Arizona,6392017,6392288,6407172.0,6472643.0,6554978.0,6632764.0,6730413.0,6829676.0,6941072.0,7044008.0,7158024.0,7278717.0,AZ
9,Arkansas,2915918,2916031,2921964.0,2940667.0,2952164.0,2959400.0,2967392.0,2978048.0,2989918.0,3001345.0,3009733.0,3017804.0,AR
10,California,37253956,37254519,37319502.0,37638369.0,37948800.0,38260787.0,38596972.0,38918045.0,39167117.0,39358497.0,39461588.0,39512223.0,CA
11,Colorado,5029196,5029319,5047349.0,5121108.0,5192647.0,5269035.0,5350101.0,5450623.0,5539215.0,5611885.0,5691287.0,5758736.0,CO
12,Connecticut,3574097,3574147,3579114.0,3588283.0,3594547.0,3594841.0,3594524.0,3587122.0,3578141.0,3573297.0,3571520.0,3565287.0,CT
13,Delaware,897934,897937,899593.0,907381.0,915179.0,923576.0,932487.0,941252.0,948921.0,956823.0,965479.0,973764.0,DE
14,District of Columbia,601723,601767,605226.0,619800.0,634924.0,650581.0,662328.0,675400.0,685815.0,694906.0,701547.0,705749.0,DC
15,Florida,18801310,18804564,18845537.0,19053237.0,19297822.0,19545621.0,19845911.0,20209042.0,20613477.0,20963613.0,21244317.0,21477737.0,FL


In [154]:
scl = [[0.0, '#00A69C'],[0.5, '#FAAF40'],[1.0, '#BE1E20']]
#scl = [[0.0,'green'],[0.5,'yellow'],[1.0,'red']]

data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019]:
    df_segmented = pd.DataFrame()
    newdict = {}
    for num in range(0,len(yeardata[year])):
        newdict[yeardata['State Code'].iloc[num]] = yeardata[year].iloc[num]
        
    df_segmented =  fulldb[fulldb['Year']== year]['State'].value_counts().to_frame()
    df_segmented.columns = ['Count']
    df_segmented['Population'] = df_segmented.index.map(newdict)
    df_segmented['Rate'] = round((df_segmented['Count']/df_segmented['Population'])*1000000,2)

    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented.index,
                        z=(df_segmented['Rate']-df_segmented['Rate'].mean())/df_segmented['Rate'].std(),
                        locationmode='USA-states',
                        colorscale = scl,
                        hovertemplate = '<b><i>State: '+ df_segmented.index.map(codetostate) + '</i></b><br><br>Death Rate Per Million People: ' + df_segmented['Rate'].astype(str) + '<extra></extra>',
                        showscale=False)

    data_slider.append(data_each_yr)
    
avgdrstate = pd.DataFrame()
avgdrstate['Col'] = np.arange(0,56)
avgdrstate.index = us_state_abbrev.values()
avgdrstate.drop('Col',inplace=True,axis=1)
for year in [2013,2014,2015,2016,2017,2018,2019]:
    newdict = {}
    for num in range(0,len(fulldb[fulldb['Year']==year]['State'].value_counts())):
        newdict[fulldb[fulldb['Year']==year]['State'].value_counts().index[num]] = fulldb[fulldb['Year']==year]['State'].value_counts().iloc[num]
    avgdrstate[str(year) + ' Count']=avgdrstate.index.map(newdict)
    newdict = {}
    for num in range(0,len(yeardata['State Code'])):
        newdict[yeardata['State Code'].iloc[num]] = yeardata[year].iloc[num]
    avgdrstate[str(year) + ' Population'] = avgdrstate.index.map(newdict)
    avgdrstate[str(year) + ' Rate'] = round(1000000*(avgdrstate[str(year) + ' Count']/avgdrstate[str(year) + ' Population']),2)
    avgdrstate.fillna(0,inplace=True)
    avgdrstate.drop([str(year) + ' Population',str(year) + ' Count'],axis=1,inplace=True)
avgdrstate['Average Rate'] = avgdrstate.apply(lambda x:x['2013 Rate']+x['2014 Rate']+x['2015 Rate']+x['2016 Rate']+x['2017 Rate']+x['2018 Rate']+x['2019 Rate'],axis=1)
avgdrstate['Average Rate'] = round(avgdrstate['Average Rate']/7,2)

data_each_yr = dict(
                        type='choropleth',
                        locations = avgdrstate.index,
                        z=(avgdrstate['Average Rate']-avgdrstate['Average Rate'].mean())/avgdrstate['Average Rate'].std(),
                        locationmode='USA-states',
                        colorscale = scl,
                        hovertemplate = '<b><i>State: '+ avgdrstate.index.map(codetostate) + '</i></b><br><br>Average Death Rate Per Million People: ' + avgdrstate['Average Rate'].astype(str) + '<extra></extra>',
                        showscale=False)

data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider)- 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
        step['args'][1][i] = True
        steps.append(step)
    else:
        step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Avg. Death Rate 2013-2019')
        step['args'][1][i] = True
        steps.append(step)

sliders = [dict(active=0, pad={"t": 0}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]

layout = dict(title ={'text':'Death Rate Per Million People by State','x':0.5,'y':0.8}, geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders,height=400,width=500)

fig13 = dict(data=data_slider, layout=layout)
fig13=go.Figure(fig13)
fig13.show()

In [15]:
popbyrace = pd.read_html('https://en.wikipedia.org/wiki/Historical_racial_and_ethnic_demographics_of_the_United_States')
popbyrace = popbyrace[8]
popbyrace['USA'] = popbyrace['USA'].apply(lambda x:int("".join(x.split())))
for cols in popbyrace.columns:
    if popbyrace[cols].dtype == 'int64':
        popbyrace[cols] = popbyrace[cols]
    else:
        popbyrace[cols] = popbyrace[cols].apply(lambda x:float(x.split('(')[1].split('%')[0]))
popbyrace['Hispanic'] = [16.40,16.67,16.87,17.08,17.34,17.58,17.76,18.07,18.3]
popbyrace.drop(['Black alone','White alone','Some other race',
       'Two or more races', 'White and Black', 'White and Asian',
       'White and American Indian and Alaska Native',
       'Black and American Indian and Alaska Native',
       'Two or more races (Other)'],inplace=True,axis=1)
for cols in popbyrace.columns:
    if 'alone' in cols:
        popbyrace[cols.split(' alone')[0]] = popbyrace[cols]
        popbyrace.drop(cols,inplace=True,axis=1)
popbyrace['American Indian and Alaska Native'].name = 'Native American'
popbyrace.columns = ['Year', 'USA', 'Hispanic', 'White', 'Black', 'Asian',
       'Native American',
       'Pacific Islander']
popbyrace.drop([0,1,2],axis=0,inplace=True)
popbyrace1 = popbyrace.copy()
for cols in popbyrace1:
    if cols != 'Year' and cols != 'USA':
        popbyrace1[cols] = popbyrace1['USA']*(popbyrace1[cols]/100)
        popbyrace1[cols] = popbyrace1[cols].astype(int)
popbyrace1['Native American'] = [5200000,5400000,6600000,6700000,6795785,6800000]
popbyrace1['Pacific Islander'] = [1200000,1300000,1400000,1500000,1400000,1400000]
popbyrace1.loc[9] = [2019,328239523,60724312,197271953,40439109,19366132,6800000,1500000]

In [156]:
data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        newdict = {}
        for cols in popbyrace1.columns:
            if cols != "Year" and cols != 'USA':
                newdict[cols] = popbyrace1[popbyrace1['Year']==year][cols].iloc[0]
        data_each_yr = dict(
                            type='pie',
                            labels = fulldb[fulldb['Year'] == year]["Victim's race"].value_counts().index,
                            values=round((fulldb[fulldb['Year'] == year]["Victim's race"].value_counts()/fulldb[fulldb['Year'] == year]["Victim's race"].value_counts().index.map(newdict))*1000000,2),
                            hovertemplate = '<b><i>Race: '+ fulldb["Victim's race"].value_counts().index +'</i></b><br><br>Death Rate Per Million People: '+ round((fulldb[fulldb['Year'] == year]["Victim's race"].value_counts()/fulldb[fulldb['Year'] == year]["Victim's race"].value_counts().index.map(newdict))*1000000,2).astype(str) +'<br>Percentage of ' + str(year)  +  ' Total: %{percent}%<extra></extra>',
                            name = year,
                            marker = dict(colors=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28','#262261'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=12,
                            outsidetextfont_size=12)
    else:
        popbyrace2 = popbyrace1.copy()
        for year1 in [2013,2014,2015,2016,2017,2018,2019]:
            newdict = {}
            for items in fulldb[fulldb['Year']==year1]["Victim's race"].value_counts().index:
                newdict[items]=fulldb[fulldb['Year']==year1]["Victim's race"].value_counts().loc[items]
            popbyrace2.iloc[year1-2013,2:] = round((popbyrace2.iloc[year1-2013,2:].index.map(newdict)/popbyrace2.iloc[year1-2013,2:])*1000000,2)
        popbyrace2.loc[10] = [np.nan,np.nan,round(popbyrace2['Hispanic'].sum()/7,2),round(popbyrace2['White'].sum()/7,2),round(popbyrace2['Black'].sum()/7,2),round(popbyrace2['Asian'].sum()/7,2),round(popbyrace2['Native American'].sum()/7,2),round(popbyrace2['Pacific Islander'].sum()/7,2)]
        newdict = {}
        for items in ['Hispanic','White','Black','Asian','Native American','Pacific Islander']:
            newdict[items] = popbyrace2.loc[10][items]
        data_each_yr = dict(
                            type='pie',
                            labels = fulldb["Victim's race"].value_counts().index,
                            values=fulldb["Victim's race"].value_counts().index.map(newdict),
                            hovertemplate = '<b><i>Race: ' + fulldb["Victim's race"].value_counts().index + '</i></b><br><br>Death Rate Per Million People: '+ fulldb["Victim's race"].value_counts().index.map(newdict).astype(str) +'<br>Percentage of Total: %{percent}%<extra></extra>',
                            name = '2013-2019 Total',
                            marker = dict(colors=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28','#262261'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=12,
                            outsidetextfont_size=12)
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Avg Death Rate Per Million People 2013-2019')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 0}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]

layout = dict(title ={'text':'Race: Death Rate Per Million People','x':0.43,'y':0.85},
              yaxis_title = {'text':'Number of Deaths'},
              sliders=sliders,height=400,width=500,showlegend=True)

figrace1 = go.Figure(dict(data=data_slider, layout=layout))

figrace1.show()

In [163]:
popbyrace3 = pd.DataFrame(np.zeros((6,6)))
popbyrace3.index = ['Hispanic','White','Black','Asian','Native American','Pacific Islander']
popbyrace3.columns = popbyrace3.index
i = 0
for items in ['Hispanic','White','Black','Asian','Native American','Pacific Islander']:
    j = 0
    for items1 in ['Hispanic','White','Black','Asian','Native American','Pacific Islander']:
        popbyrace3.iloc[i,j] = round(popbyrace2.loc[10][items]/popbyrace2.loc[10][items1],2)
        j = j+1
    i = i+1
    
popbyrace4 = popbyrace3.copy()
for cols in popbyrace4:
    for num in range(0,len(popbyrace4)):
        if popbyrace4[cols].iloc[num] == 1.0:
            popbyrace4[cols].iloc[num] = 'just'
            
            
figraceheat = go.Figure().add_trace(go.Heatmap(z=popbyrace3,
                       x = ['Hispanic','White','Black','Asian','Native American','Pacific Islander'],
                       y = ['Hispanic','White','Black','Asian','Native American','Pacific Islander'],
                       colorscale=[[0.0, '#262261'],[0.25, '#00A69C'],[0.5, '#FAAF40'],[0.75,'#F05A28'],[1.0,'#BE1E20']],
                       showscale = False,
                       hovertemplate = '<b><i>Death Rate Comparison: </i></b><br><br>%{y} people are %{z}x as likely to be killed as %{x} people<extra></extra>'))
figraceheat.update_layout(title={'text':"Death Rate Comparison by Race",'x':0.5,'y':0.85},width=400,height=400)
figraceheat.show()

In [164]:
#GENDER RATE
l1 = [2013,2014,2015,2016,2017,2018,2019]
male = [155000000,156120000,157260000,158380000,159410000,160400000,161480000]
female = [161330000,162500000,163670000,164840000,165920000,166950000,168080000]
yearpopgender = pd.DataFrame()
yearpopgender['Male'] = male
yearpopgender['Female'] = female
yearpopgender.index = l1
yearpopgender

for years in l1:
    for gender in ['Male','Female']:
        yearpopgender.loc[years,gender] = round((len(fulldb[(fulldb['Year'] == years) & (fulldb["Victim's gender"] == gender)])/yearpopgender.loc[years,gender])*1000000,2)
        
yearpopgender.loc['2013-2019 Average Death Rate Per Million People'] = [round(yearpopgender['Male'].sum()/7,2),round(yearpopgender['Female'].sum()/7,2)]
yearpopgender['Text'] = round(yearpopgender['Male']/yearpopgender['Female'],2)
yearpopgender['Text'] = yearpopgender['Text'].apply(lambda x: 'Males were ' + str(x) + 'x as likely to be killed as Females')
yearpopgender

data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        data_each_yr = dict(
                            type='pie',
                            labels = yearpopgender.columns[:2],
                            values=yearpopgender.loc[year][['Male','Female']],
                            hovertemplate = '<b><i>Sex: ' + yearpopgender.columns[:2] +'</i></b><br><br>Death Rate Per Million People: ' + yearpopgender.loc[year][['Male','Female']].astype(str) + '<br>' + yearpopgender.loc[year,'Text']  +  '<extra></extra>',
                            name = year,
                            marker = dict(colors=['#FAAF40','BE1E20'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=14,
                            outsidetextfont_size=14
                            )
    else:
        data_each_yr = dict(
                            type='pie',
                            labels = yearpopgender.columns[:2],
                            values=yearpopgender.loc['2013-2019 Average Death Rate Per Million People'][['Male','Female']],
                            hovertemplate = '<b><i>Sex: ' + yearpopgender.columns[:2] +'</i></b><br><br>Death Rate Per Million People: ' + yearpopgender.loc['2013-2019 Average Death Rate Per Million People'][['Male','Female']].astype(str) + '<br>' + yearpopgender.loc['2013-2019 Average Death Rate Per Million People','Text']  +  '<extra></extra>',
                            name = '2013-2019 Average Death Rate Per Million People',
                            marker = dict(colors=['#FAAF40','BE1E20'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=14,
                            outsidetextfont_size=14)
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Average Death Rate Per Million People')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 0}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]


layout = dict(title ={'text':'Sex: Death Rate Per Million People','x':0.47,'y':0.85},
              yaxis_title = {'text':'Death Rate'},
              sliders=sliders,height=400,width=400,showlegend=True)

figsex1 = go.Figure(dict(data=data_slider, layout=layout))

figsex1.show()

In [59]:
data = pd.read_html('https://en.wikipedia.org/wiki/Demographics_of_the_United_States')
data = data[1][['age (years)','total']]
data = data.iloc[:5]

In [71]:
ages = [0,14,24,54,64]
data1 = pd.DataFrame()
for year in [2013,2014,2015,2016,2017,2018,2019]:
    for num in range(0,len(data['age (years)'])):
        if num == 0:
            data1.loc[year,data['age (years)'].iloc[num]] = round(1000000*(len(fulldb[(fulldb['Year'] == year) & (fulldb["Victim's age"] <= 14)])/data['total'].iloc[num]),2)
        elif num < 3:
            data1.loc[year,data['age (years)'].iloc[num]] = round(1000000*(len(fulldb[(fulldb['Year'] == year) & (fulldb["Victim's age"] <= ages[num + 1]) & (fulldb["Victim's age"] >= ages[num] + 1)])/data['total'].iloc[num]),2)
        else:
            data1.loc[year,data['age (years)'].iloc[num]] = round(1000000*(len(fulldb[(fulldb['Year'] == year) & (fulldb["Victim's age"] >= 65)])/data['total'].iloc[num]),2)
data

In [74]:
newlist = [round(data1[x].sum()/7,2) for x in data1.columns]

In [77]:
data1.loc['2013-2019 Average Death Rate Per Million People'] = newlist

In [78]:
data1

,0–14,15–24,25–54,55–64,65+
2013,0.07,4.80,4.66,0.71,0.59
2014,0.10,4.54,5.30,0.69,0.57
2015,0.03,4.57,6.01,0.57,0.47
2016,0.07,4.06,5.84,0.64,0.53
2017,0.03,4.29,5.70,0.73,0.61
2018,0.03,4.01,5.15,0.73,0.61
2019,0.02,3.16,5.04,0.47,0.39
2013-2019 Average Death Rate Per Million People,0.05,4.20,5.39,0.65,0.54


In [166]:
data_slider = []
for year in [2013,2014,2015,2016,2017,2018,2019,'Total']:
    if year != 'Total':
        data_each_yr = dict(
                            type='pie',
                            labels = data1.columns,
                            values=data1.loc[year],
                            hovertemplate = '<b><i>Age Group: ' + data1.columns +'</i></b><br><br>Death Rate Per Million People: ' + data1.loc[year].astype(str) + '<extra></extra>',
                            name = year,
                            marker = dict(colors=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=14,
                            outsidetextfont_size=14
                            )
    else:
        data_each_yr = dict(
                            type='pie',
                            labels = data1.columns,
                            values=data1.loc['2013-2019 Average Death Rate Per Million People'],
                            hovertemplate = '<b><i>Age Group: ' + data1.columns +'</i></b><br><br>Death Rate Per Million People: ' + data1.loc['2013-2019 Average Death Rate Per Million People'].astype(str) + '<extra></extra>',
                            name = '2013-2019 Average Death Rate Per Million People',
                            marker = dict(colors=['#FAAF40','#BE1E20','#00A69C','#894BAA','#F05A28'],line_color='black',line_width=2),
                            textinfo = 'none',
                            insidetextfont_size=14,
                            outsidetextfont_size=14)
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    if i < len(data_slider) - 1:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2013))
    else:
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='2013-2019 Average Death Rate Per Million People')
    step['args'][1][i] = True
    steps.append(step)
    

sliders = [dict(active=0, pad={"t": 0}, steps=steps,bgcolor='#FAAF40',bordercolor='black',activebgcolor='#BE1E20')]

layout = dict(title ={'text':'Age Group: Death Rate Per Million People','x':0.47,'y':0.85},
              yaxis_title = {'text':'Death Rate'},
              sliders=sliders,height=400,width=400,showlegend=True)

figages = go.Figure(dict(data=data_slider, layout=layout))

figages.show()

In [165]:
data2 = pd.DataFrame(np.zeros((5,5)))
for num in range(0,len(data1.loc['2013-2019 Average Death Rate Per Million People'])):
    for num1 in range(0,len(data1.loc['2013-2019 Average Death Rate Per Million People'])):
        data2.iloc[num,num1] = round(data1.loc['2013-2019 Average Death Rate Per Million People'].iloc[num]/data1.loc['2013-2019 Average Death Rate Per Million People'].iloc[num1],2)
data2.index = data1.columns
data2.columns = data1.columns

figageheat = go.Figure().add_trace(go.Heatmap(z=data2,
                       x = data2.columns,
                       y = data2.columns,
                       colorscale=[[0.0, '#262261'],[0.25, '#00A69C'],[0.5, '#FAAF40'],[0.75,'#F05A28'],[1.0,'#BE1E20']],
                       showscale = False,
                       hovertemplate = '<b><i>Death Rate Comparison: </i></b><br><br>People in %{y} Age Group are %{z}x as likely to be killed as people in %{x} Age Group<extra></extra>'))
figageheat.update_layout(title={'text':"Death Rate Comparison by Age",'x':0.5,'y':0.85},width=400,height=400)
figageheat.show()

In [171]:
html_div10 = plot(fig13,output_type='div')
html_div11 = plot(figrace1,output_type='div')
html_div12 = plot(figraceheat,output_type='div')
html_div13 = plot(figsex1,output_type='div')
html_div14 = plot(figages,output_type='div')
html_div15 = plot(figageheat,output_type='div')
Html_file= open("policeviolence3.html","w")
Html_file.write('<div style="overflow-x:auto;"><table style="margin-left:auto;margin-right:auto"><tr><td>' + html_div11 + '</td><td>' + html_div13 + '</td><td>' + html_div14 + '</td></tr></table></div><div style="overflow-x:auto;"><table style="margin-left:auto;margin-right:auto"><tr><td>'+html_div12+'</td><td>' + html_div10 + '</td><td>' + html_div15+ '</td></tr></table></div>')

20079945

In [170]:
html_div = plot(figrace,output_type='div')
html_div1 = plot(figage,output_type='div')
html_div2 = plot(figsex,output_type='div')
html_div3 = plot(fig3,output_type='div')
html_div4 = plot(figracesex,output_type='div')
html_div5 = plot(figraceage,output_type='div')
html_div6 = plot(figagesex,output_type='div')
Html_file= open("policeviolence2.html","w")
Html_file.write('<div style="overflow-x:auto;"><table style="margin-left:auto;margin-right:auto"><tr><td>' + html_div + '</td><td>' + html_div2 + '</td><td>' + html_div1 + '</td></tr></table></div><div style="overflow-x:auto;"><table style="margin-left:auto;margin-right:auto"><tr><td>'+html_div5+'</td><td>' + html_div3 + '</td></tr><tr><td>' + html_div4+ '</td><td>'+html_div6+'</td></tr></table></div>')
Html_file.close()

In [182]:
import codecs
file = codecs.open("policeviolence2.html", "r", "utf-8")
import pyperclip
pyperclip.copy(file.read())
spam = pyperclip.paste()